In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import requests 
from bs4 import BeautifulSoup 

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [2]:
CLIENT_ID = 'UXWV2V2VG3VGA3O10HEBJIKLMVERAKIVO45VDRHIA0XBNOJH' # your Foursquare ID
CLIENT_SECRET = 'STQCNT0UM1BMGR354CNXQU4P035Y3RU0AABKXK4TPNODES0R' # your Foursquare Secret
ACCESS_TOKEN = 'PBNLWQG4HXOXBIX2BPN0FB42X20O5TCPZET4WOKFGPHB1OGM' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UXWV2V2VG3VGA3O10HEBJIKLMVERAKIVO45VDRHIA0XBNOJH
CLIENT_SECRET:STQCNT0UM1BMGR354CNXQU4P035Y3RU0AABKXK4TPNODES0R


In [3]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(URL) 
  
soup = BeautifulSoup(page.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

# print(soup.prettify()) 
print('Page Scrapped.')

Page Scrapped.


In [5]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df = pd.read_html(page.content, header=0) [0]
df.rename(index=str, columns={"Postal Code": "PostalCode"}, inplace = True)
df.rename(index=str, columns={"Neighbourhood": "Neighborhood"}, inplace= True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df_2 = df[df.Borough != 'Not assigned'].reset_index(drop=True) 
df_2.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [7]:
#More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: 
#Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.

df_3 = df_2.groupby(['PostalCode','Borough'], as_index=False).agg(lambda x: ','.join(x))
df_3.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
replace=df_3['Neighborhood']=="Not assigned"
df_3.loc[replace, 'Neighborhood']=df_3.loc[replace, 'Borough']
df_3.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
df_3.shape

(103, 3)

In [10]:
path= 'http://cocl.us/Geospatial_data'
data=pd.read_csv(path)
data.rename(index=str, columns={"Postal Code": "PostalCode"}, inplace = True)
data.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


*used CSV file* <br>
~Google Maps Geocoding API~

In [11]:
data2=pd.merge(df_3, data)

data2
#Combined both dataframes

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [ ]:
<i>